# Problem 1.1

In [1]:
import json
import matplotlib.pyplot as plt
import datetime, time
import pytz
from datetime import timedelta


#================================= Plot the Histogram ==============================
def plot_histogram(time_stamps, hashtag):
    hourly_tweet_count = [0] * (int((max(time_stamps) - min(time_stamps)) / timedelta(hours=1))+1)
    start_time = min(time_stamps)
    for time_stamp in time_stamps:
        hourly_tweet_count[int((time_stamp-start_time) / timedelta(hours=1))] += 1
           
#     plt.figure(figsize = (12,8))
#     plt.bar([i for i in range(0,len(hourly_tweet_count))], hourly_tweet_count, 1, align='edge', color = 'k')
#     hours_passed = (max(time_stamps) - min(time_stamps)) / timedelta(hours=1)
#     plt.axis([0,hours_passed+ 1,0,int(max(hourly_tweet_count)*1.03)])
    plt.figure(figsize = (12,8))
    plt.plot(hourly_tweet_count)
    plt.xlabel('Time(Hour)', fontsize = 15)
    plt.ylabel('Number of Tweets', fontsize = 15)
    plt.title('Number of Tweets over Time (' + hashtag + ')', fontsize = 20)
    plt.show()        
#==============================================================================


#=================== Get required Information =================
hashtag_dict = {'#GoHawks' : 'tweets_#gohawks.txt',
                '#GoPatriots' : 'tweets_#gopatriots.txt',
                '#NFL' : 'tweets_#nfl.txt',
                '#Patriots' : 'tweets_#patriots.txt',
                '#SB49' : 'tweets_#sb49.txt',
                '#SuperBowl' : 'tweets_#superbowl.txt'}




In [4]:
import copy
import numpy as np

input_file = open('./tweet_data/tweets_#gohawks.txt' )
def input_regr(file, time_flag, folder):
    pst_tz = pytz.timezone('US/Pacific')
    timecut1 = pst_tz.localize(datetime.datetime(2015,2,1,8,0))
    timecut2 = pst_tz.localize(datetime.datetime(2015,2,1,21,0))
    dict1 = dict([])
    dict2 = dict([])
    dict3 = dict([])
    
    ipt = open(folder + file)   
    for line in ipt:
        data = json.loads(line)
        date_time = datetime.datetime.fromtimestamp(data['citation_date'], pst_tz)
        time_key = str(date_time.month) + ' ' + str(date_time.day) + ' ' + str(date_time.hour)

        dict_cur = dict1               
        if time_flag and date_time >= timecut2:
            dict_cur = dict3
        elif time_flag and date_time >= timecut1:
            dict_cur = dict2   

        if time_key not in dict_cur:
            dict_cur[time_key] = [0,0,0,0,0]
            dict_cur[time_key][4] = date_time.hour
        dict_cur[time_key][0] = dict_cur[time_key][0] + 1
        dict_cur[time_key][1] = dict_cur[time_key][1] + int(data['metrics']['citations']['total'])
        dict_cur[time_key][2] = dict_cur[time_key][2] + int(data['author']['followers'])
        dict_cur[time_key][3] = max(dict_cur[time_key][3], int(data['author']['followers']))
    ipt.close()
    
    res1 = [dict1[k] for k in dict1]
    res2 = [dict2[k] for k in dict2]
    res3 = [dict3[k] for k in dict3]    
    return [res1, res2, res3] 
        

In [9]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

def cross_valid(x_data, y_data, f):    
    kf = KFold(n_splits=10)
    kf.get_n_splits(x_data)    
    error = []    
    for train_idx, test_idx in kf.split(x_data):
        regr = f.fit(x_data[train_idx], y_data[train_idx])
        pred = regr.predict(x_data[test_idx])
        error.append(np.sum(np.abs(y_data[test_idx] - pred)))  
    return (np.sum(np.array(error))) / len(y_data)

#when I do the basic linear regression here, I ignore number if tweets here
#,,,not sure
def get_x_y(data, time_diff):    
    data = np.array(data)
    y = data[time_diff:len(data), 0]
    x = data[0:, 1:]
    for i in range((len(data) - time_diff)):
        for j in range(time_diff - 1):
            x[i][0] += x[i + j][0]
            x[i][1] += x[i + j][1]
            x[i][2] = max(x[i][2], x[i + j][2])
    x = x[0:(len(data) - time_diff),:]
    return x, y

all_data = []
for key in hashtag_dict:
    print('\n' + key)
    r = input_regr(hashtag_dict[key], 1, './tweet_data/') 
    all_data.append(r)
    for i in range(3):
        x, y = get_x_y(r[i], 1)
        print(cross_valid(x,y, LinearRegression()))
    
#try some transformations



#SB49
150.5588870716524
74829.18785219172
449.473083640796

#GoHawks
282.5089304764196
4897.045409810998
63.47644405764414

#NFL
210.14342695041668
4705.738936971248
261.22056644489044

#SuperBowl
500.87892420934674
217697.46646345916
654.7042266053791

#Patriots
309.9998655781271
21968.50682678939
205.32316167691945

#GoPatriots
21.253749166950666
2136.5506403565964
19.56016215061979


In [22]:
#put the date from all tags together and generate conrresponding a and y
#data--original data
#k-----whether the time span has been splitted into k parts, k = 1 or 3
#diff--the time diff
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn import neighbors

def vstack_data(data, k, diff):
    x = []
    y = []
    for item in all_data:
        for i in range(k):
            x_tmp, y_tmp = get_x_y(item[i], diff)
            if len(x) < i + 1:
                x.append(x_tmp)
                y.append(y_tmp)
                continue
            x[i] = np.vstack((x[i], x_tmp))
            y[i] = np.hstack((y[i], y_tmp))
    return x, y

x, y = vstack_data(all_data, 3, 1)    
    
print('\nlinear regression')
for i in range(3):
    print(cross_valid(x[i],y[i], LinearRegression()))
    
print('\nRamdom forest')
for i in range(3):
    forest = RandomForestRegressor(n_estimators = 1000,criterion='mse',random_state=1,n_jobs=-1)
    print(cross_valid(x[i],y[i], forest))

print('\nSVM')
for i in range(3):
    svr = svm.SVR()
    print(cross_valid(x[i],y[i], svr))

print('\nKNN')
for i in range(3):
    knn = neighbors.KNeighborsRegressor()
    print(cross_valid(x[i],y[i], knn))
    
print('\ncombined log transformation and squared transformation for time')
for i in range(3):
    x_new = copy.deepcopy(x[i])
    x_new[:,1] = np.log(x_new[:,1] + 1)
    x_new[:,2] = np.log(x_new[:,2] + 1)
    squared = x_new[:,3] ** 2
    x_new = np.insert(x_new, 1, values=squared, axis=1)
    print(cross_valid(x_new,y[i], LinearRegression()))
    


linear regression
279.0621562710869
34627.90262937886
388.199220305292

Ramdom forest
297.8232232741618
28394.08926388889
359.83636493370955

SVM
238.42731276415077
26727.51388888889
350.2940439525644

KNN
316.5681262327416
30281.847222222223
369.557225433526

combined log transformation and squared transformation for time
268.67718244212597
30574.82444471826
359.4423338396541


In [16]:
#extract test files
files = []
for i in range(10):    
    j = i + 1
    if i == 3: 
        j = 1
    elif i > 3 and i < 7:
        j = i - 3
    elif i >= 7:
        j = i - 6
    name = 'sample' + str(i + 1) + '_' + 'period' + str(j) + '.txt'
    files.append(name)

r_test = [input_regr(file, 0, './test_data/') for file in files] 
all_data_new = []
for key in hashtag_dict:
    r = input_regr(hashtag_dict[key], 0, './tweet_data/') 
    all_data_new.append(r)

In [25]:

i_const = 5
x, y = vstack_data(all_data_new, 1, 5)
f = RandomForestRegressor(n_estimators = 1000,criterion='mse',random_state=1,n_jobs=-1)
regr = f.fit(x[0], y[0])
print(regr.estimators_ , regr.feature_importances_ )
idx = 0
for t in r_test:
    t = t[0]
    i = min(i_const, len(t) - 1)
    x_test = copy.deepcopy(np.array([0,0,0,t[- i - 1][4]]))
    y_test = t[-1][0]
    for j in range(i):
        x_test[0] += t[- j - 2][1]
        x_test[1] += t[- j - 2][2]
        x_test[2] = max(x_test[2], t[- j - 2][3])
    pred = regr.predict(x_test.reshape(1,-1))
    print(files[idx], pred, np.abs(y_test - pred[0])) 
    idx = idx + 1

[DecisionTreeRegressor(criterion='mse', max_depth=None, max_features='auto',
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=1791095845, splitter='best'), DecisionTreeRegressor(criterion='mse', max_depth=None, max_features='auto',
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=2135392491, splitter='best'), DecisionTreeRegressor(criterion='mse', max_depth=None, max_features='auto',
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=946286476, splitter='best'), DecisionTreeRegressor(criterion='m

In [26]:
i_const = 5
x, y = vstack_data(all_data_new, 1, 5)
f = svm.SVR()
regr = f.fit(x[0], y[0])
idx = 0
for t in r_test:
    t = t[0]
    i = min(i_const, len(t) - 1)
    x_test = copy.deepcopy(np.array([0,0,0,t[- i - 1][4]]))
    y_test = t[-1][0]
    for j in range(i):
        x_test[0] += t[- j - 2][1]
        x_test[1] += t[- j - 2][2]
        x_test[2] = max(x_test[2], t[- j - 2][3])
    pred = regr.predict(x_test.reshape(1,-1))
    print(files[idx], pred, np.abs(y_test - pred[0])) 
    idx = idx + 1

sample1_period1.txt [104.125] 72.875
sample2_period2.txt [104.125] 19692.875
sample3_period3.txt [104.125] 513.875
sample4_period1.txt [104.125] 313.875
sample5_period1.txt [104.125] 236.875
sample6_period2.txt [104.125] 41.875
sample7_period3.txt [104.125] 2.125
sample8_period1.txt [104.125] 93.125
sample9_period2.txt [104.125] 1750.875
sample10_period3.txt [104.125] 43.125


# Problem 3

In [35]:
from sklearn.feature_extraction import text
def extract_tweets(hashtag_dict, folder):
    titles = []
    times = []
    location = []
    for key in hashtag_dict:
        ipt = open(folder + hashtag_dict[key])   
        for line in ipt:
            data = json.loads(line)
            titles.append(data['title'])
            times.append(data['citation_date'])
            loc = data['tweet']['user']['location']
            if loc == 'Washington' or ', Washington' in loc or loc == 'WA' or ', WA' in loc:
                location.append(-1)
            elif loc == 'Massachusetts' or ', Massachusetts' in loc or loc == 'MA' or ', MA' in loc:
                location.append(1)
            else:
                location.append(0)
    return titles, times, location
titles, times, locations = extract_tweets(hashtag_dict, './tweet_data/')    
    

In [58]:
pst_tz = pytz.timezone('US/Pacific')
time_slices = []
for i in range(14):
    time_slices.append(pst_tz.localize(datetime.datetime(2015,2,1,i + 8,0)))
    
def split_time(time_slices, titles, times, locations):
    splitted_titles = []
    splitted_locations = []
    for i in range(len(time_slices) - 1):
        splitted_titles.append([])
        splitted_locations.append([])
    for i in range(len(times)):
        date_time = datetime.datetime.fromtimestamp(times[i], pst_tz)
        for j in range(len(time_slices) - 1):
            if date_time >= time_slices[j] and date_time <= time_slices[j + 1]:
                splitted_titles[j].append(titles[i]) 
                splitted_locations[j].append(locations[i]) 
    return splitted_titles, splitted_locations
    
splitted_titles, splitted_locations = split_time(time_slices, titles, times, locations)    

In [8]:
def matrix2tfidf(matrix):
    count_vect = text.CountVectorizer(min_df=3, stop_words='english')
    X_train_counts = count_vect.fit_transform(matrix)
    tfidf_transformer = text.TfidfTransformer()
    tfidf = tfidf_transformer.fit_transform(X_train_counts)
    return tfidf, count_vect


In [60]:
from sklearn.decomposition import NMF
nmf_model = NMF(n_components=2, init='random', random_state=0)
for i in range(len(splitted_titles)):
    item = splitted_titles[i]
    loc = splitted_locations[i]
    print(time_slices[i])
    tfidf, count_vect = matrix2tfidf(item)
    
    #check tweets
    a11 = 0
    a12 = 0
    a21 = 0
    a22 = 0
    line_num = 0   
    tweet_dis = nmf_model.fit_transform(tfidf)
    for line in tweet_dis:
        if loc[line_num] == 1:
            left += 1
        elif loc[line_num] == -1:
            right += 1
        if line[0] > line[1]: 
            if loc[line_num] == 1:
                a11 += 1
            elif loc[line_num] == -1:
                a12 += 1
        else:
            if loc[line_num] == 1:
                a21 += 1
            elif loc[line_num] == -1:
                a22 += 1
        line_num += 1
    print(a11, a12)
    print(a21, a22)
    
    #check words
    word_dis = nmf_model.components_
    word_sorted = np.argsort(-word_dis, axis = 1)[:,:10]
    words = count_vect.get_feature_names()
    for line in word_sorted:
        for k in line:
            print(words[k],' ' ,end = '')
        print('\n')

2015-02-01 08:00:00-08:00
68 22
76 136
superbowl  http  sunday  happy  ready  nfl  today  game  football  day  

sb49  seahawks  patriots  super  bowl  gohawks  nfl  superbowlxlix  today  http  

2015-02-01 09:00:00-08:00
59 295
143 73
sb49  seahawks  patriots  gohawks  game  http  day  superbowlxlix  today  win  

superbowl  http  sunday  nfl  super  bowl  happy  today  ready  football  

2015-02-01 10:00:00-08:00
80 637
505 434
seahawkswin  seahawks  winning  ve  got  nfl  sb49  http  superbowlxlix  superbowl  

patriotswin  patriots  winning  ve  got  nfl  sb49  http  superbowl  superbowlxlix  

2015-02-01 11:00:00-08:00
1462 514
111 1038
patriotswin  patriots  ve  winning  got  nfl  sb49  http  superbowlxlix  superbowl  

seahawkswin  seahawks  ve  winning  got  nfl  sb49  http  gohawks  superbowl  

2015-02-01 12:00:00-08:00
1315 542
80 1146
patriotswin  patriots  ve  winning  got  nfl  sb49  http  superbowlxlix  superbowl  

seahawkswin  seahawks  ve  winning  got  nfl  sb49  htt

In [51]:
a = [i for loc in locations if loc == 1]
len(a)

42988

In [52]:
left = 0
for line in locations:

61301